In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Mendefinisikan direktori data pelatihan dan validasi
train_dir = '/content/drive/MyDrive/Klasifikasi Anjing/Dog Breed/Dog 5 Kelas/Train'
validation_dir = '/content/drive/MyDrive/Klasifikasi Anjing/Dog Breed/Dog 5 Kelas/Tes'

In [ ]:
img_size  = 224
batch_size = 32

In [ ]:
# Mengkonversi gambar palet menjadi RGBA
def preprocess_image(image):
    if image.mode == 'P':
        image = image.convert('RGBA')
    image = image.resize((img_size, img_size))
    return image
    
image_gen_train = ImageDataGenerator(rescale = 1./255)

train_datagen = image_gen_train.flow_from_directory(batch_size = batch_size, 
                                                     directory = train_dir,
                                                     shuffle= True,
                                                     target_size = (img_size,img_size),
                                                     class_mode = 'categorical')

image_generator_validation = ImageDataGenerator(rescale=1./255)

val_datagen = image_generator_validation.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(img_size, img_size),
                                                              class_mode='categorical')

In [ ]:
pre_trained_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

In [ ]:
for layer in pre_trained_model.layers:
    print(layer.name)
    layer.trainable = False

In [ ]:
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output
x = tf.keras.layers.GlobalMaxPooling2D()(last_output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(5, activation='softmax')(x)

In [ ]:
model = tf.keras.Model(pre_trained_model.input, x)

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.categorical_crossentropy, metrics=['acc'])

In [ ]:
model.summary()

In [ ]:
vgg_classifier = model.fit(train_datagen,
                           steps_per_epoch=train_datagen.samples // batch_size,
                           epochs = 30, 
                           validation_data=val_datagen,
                           validation_steps=val_datagen.samples // batch_size,
                           verbose = 1)

In [ ]:
acc      = vgg_classifier.history[ 'acc' ]
val_acc  = vgg_classifier.history[ 'val_acc' ]
loss     = vgg_classifier.history[ 'loss' ]
val_loss = vgg_classifier.history[ 'val_loss' ]

epochs   = range(len(acc))

plt.plot  ( epochs, acc, label='Training')
plt.plot  ( epochs, val_acc, label='Validation')
plt.title ('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot  ( epochs, loss, label='Training')
plt.plot  ( epochs, val_loss, label='Validation')
plt.legend()
plt.title ('Training and validation loss')



In [ ]:
# Simpan model ke dalam file .h5
model.save('/content/drive/MyDrive/Klasifikasi Anjing/Dog Breed/Dog 5 Kelas/Klasifikasi 5 Ras Anjing.h5')

In [ ]:
# Mengonversi model menjadi JSON
model_json = model.to_json()

# Menyimpan model JSON ke dalam file
with open('/content/drive/MyDrive/Klasifikasi Anjing/Dog Breed/Dog 5 Kelas/Klasifikasi 5 Ras Anjing.json', 'w') as json_file:
    json_file.write(model_json)